In [ ]:
import os
import sys
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import argparse
import torch
import torch.nn.init as init
from transformers import (
    AutoTokenizer, LlamaConfig, LlamaForCausalLM, TrainingArguments, Trainer, PreTrainedTokenizerFast
)
from datasets import Dataset, DatasetDict
import argparse
import pandas as pd
from huggingface_hub import hf_hub_download

repo = "ai4bharat/sangraha"
sb_folder = "verified/tel"
local_dir = "ai4bharat"

In [19]:
def get_size(df):
    return sys.getsizeof(df) / 1024 / 1024

In [ ]:
def get_dataset(repo, sb_folder, local_dir, range_):
    telugu_char_pattern = re.compile(r'[^\u0C00-\u0C7F\u0000-\u007F\s<>\n0-9.,!?]')
    filenames = [f"data-{i}.parquet" for i in range(*range_)]
    df_dataset = pd.DataFrame()
    for filename in filenames:
        file_path = hf_hub_download(
            repo_id=repo,
            repo_type="dataset",
            subfolder=sb_folder,
            filename=filename,
            local_dir=local_dir
        )
        df = pd.read_parquet(file_path)
        df = df.rename(columns={"text": "Input"})
        df.drop(columns=['type'], inplace=True)
        df.drop(columns=['doc_id'], inplace=True)
        df["Input"] = "<bos> " + df["Input"] + " <eos>"
        df["Input"] = df["Input"].str.replace("\n", "<newline>")
        df["Input"] = df["Input"].apply(lambda x: telugu_char_pattern.sub("", x))
        df_dataset = pd.concat([df_dataset, df], ignore_index=True)
    
    return df_dataset

In [13]:
df = get_dataset(repo, sb_folder, local_dir, (13, 14))
df

,Input
0,<bos> వెండితెరపై లవర్ బాయ్స్ చాలామంది ఉన్నారు....
1,<bos> ఎల్ఐసీ పాలసీని మధ్యలోనే ఆపేశారా?<newline...
2,<bos> సరికాని ఆహారం మరియు ఉత్పత్తుల నాణ్యమైన న...
3,<bos> మహేష్ సినిమాలో విజయశాంతి పాత్ర ఇదే!<newl...
4,<bos> ఢిల్లీ : భారతీయ రైల్వే మరో ఘనతను సాధించి...
...,...
174757,"<bos> ఎ. వెంక టేశ్వరరావు (విజయవాడ), ఎస్. సత్యన..."
174758,<bos> భారత ప్రధాని హోదాలో అగ్రరాజ్యం అమెరికా ప...
174759,<bos> నవతరం దర్శకుల్లో తనదైన అభిరుచిని చాటుకుం...
174760,<bos> తన భార్య అనుష్క డెలివరీ నేపథ్యంలో విరాట్...


In [ ]:
dataset_telugu = Dataset.from_pandas(df)

In [15]:
tokenizer = AutoTokenizer.from_pretrained("SentencePieceBPETokenizer_500")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["Input"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = df.map(tokenize_function, batched=True)

In [ ]:
llama_config = LlamaConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=512,
    num_hidden_layers=6,
    num_attention_heads=8,
    intermediate_size=2048,
    max_position_embeddings=512,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

model = LlamaForCausalLM(config=llama_config)

In [ ]:
for i,j in model.named_parameters():
  if j.requires_grad and len(j.size()) > 1:
    init.xavier_uniform_(j.data)
    
total_param=0
for i,j in model.named_parameters():
    total_param += j.numel()
print(total_param/(10**6))

In [ ]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

training_args = TrainingArguments(
    output_dir="./llama-telugu",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=100,
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    perplexity = torch.exp(torch.tensor(eval_pred[1].mean()))
    return {"perplexity": perplexity.item()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

perplexity_log = []
for epoch in range(1, 4):
    results = trainer.evaluate()
    perplexity = results["eval_perplexity"]
    perplexity_log.append((epoch, perplexity))
    print(f"Epoch: {epoch}, Perplexity: {perplexity}")

trainer.save_model("./llama-telugu")

In [ ]:
prompts = [
    "తెలుగు భాష అనేది",
    "ఇది ఒక గొప్ప",
    "మీ పేరేమిటి?",
    "నేను ఎలా చేయగలను",
    "కళలు మన",
    "పుస్తకం చదివితే",
    "సాహిత్యం అంటే",
    "ప్రపంచానికి తెలుగు",
    "తెలుగులో గొప్ప రచనలు",
    "భావన నిండిన"
]

for prompt in prompts:
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    output = model.generate(input_ids, max_length=50, num_return_sequences=1, do_sample=True, temperature=0.7)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Prompt: {prompt}\nResponse: {response}\n")


In [ ]:
perplexity_df = pd.DataFrame(perplexity_log, columns=["Epoch", "Perplexity"])
print(perplexity_df)